In [4]:
import numpy as np

In [5]:
transition_probabilities = [  # shape=[s, a, s']
                                [[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]],
                                [[0.0, 1.0, 0.0], None, [0.0, 0.0, 1.0]],
                                [None, [0.8, 0.1, 0.1], None]
                            ]
rewards = [  # shape=[s, a, s']
            [[+10, 0, 0], [0, 0, 0], [0, 0, 0]],
            [[0, 0, 0], [0, 0, 0], [0, 0, -50]],
            [[0, 0, 0], [+40, 0, 0], [0, 0, 0]]
        ]
 # shape = [s]
possible_actions = [[0, 1, 2], [0, 2], [1]]

In [ ]:
def step(state,action):
    proba = transition_probabilities[state][action]
    next_state = 
    reward = rewards[state][action][next_state]

In [ ]:
!pip list

In [ ]:
import numpy as np